<a href="https://colab.research.google.com/github/cojocarucosmin/Optimizers/blob/main/Optimal_Delivery_Path_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://account.heigit.org/signup

In [ ]:
!pip install openrouteservice

In [ ]:
import openrouteservice
import numpy as np
from google.colab import userdata
# Your ORS API key (get one free at https://openrouteservice.org/sign-up/)

ORS_API_KEY = userdata.get("ORS_API_KEY")
client = openrouteservice.Client(key=ORS_API_KEY)

# Your hardcoded locations (lon, lat)
locations = [
    (25.9696, 44.4481),  # Strada Preciziei 24, Chiajna
    (26.1025, 44.4268),  # Piața Unirii
    (26.0122, 44.4352),  # Bulevardul Iuliu Maniu 220
    (26.1097, 44.4781),  # Strada Barbu Văcărescu 201
    (25.9987, 44.4260),  # Strada Drumul Taberei 34
    (26.1457, 44.3755),  # Șoseaua Olteniței 103, Popești-Leordeni
    (26.2032, 44.4547),  # Strada 1 Decembrie 1918 12, Pantelimon
    (26.1427, 44.5076),  # Strada Erou Iancu Nicolae 67, Voluntari
    (25.9692, 44.3972)   # Prelungirea Ghencea 85, Domnești
]

# ORS expects (lon, lat)
coords = locations

# Get distance and duration matrices for a truck
matrix = client.distance_matrix(
    locations=coords,
    profile='driving-hgv',  # heavy goods vehicle (truck)
    metrics=['distance', 'duration'],
    units='km'
)

distance_matrix = np.array(matrix['distances'])
duration_matrix = np.array(matrix['durations'])

print("Distance matrix (km):")
print(distance_matrix)
print("Duration matrix (minutes):")
print(duration_matrix / 60)

Distance matrix (km):
[[ 0.   14.4   5.24 16.    6.01 20.3  25.84 24.36  9.69]
 [13.49  0.    9.75  7.56 10.28  8.49 12.   11.78 13.86]
 [ 6.02  9.83  0.   11.71  2.33 15.74 21.28 19.63  8.53]
 [16.36  8.59 12.61  0.   14.22 14.75 10.77  5.14 19.61]
 [ 6.88  9.82  2.87 14.22  0.   15.73 21.27 17.98  7.3 ]
 [20.55  8.49 16.33 13.69 16.1   0.   18.12 17.49 21.81]
 [24.36 11.27 20.61 13.7  21.55 18.21  0.   22.25 49.81]
 [26.67 11.18 17.4   5.17 19.01 18.54 20.87  0.   32.49]
 [10.52 13.67  8.72 19.88  7.18 22.05 50.67 30.51  0.  ]]
Duration matrix (minutes):
[[ 0.         25.7035     11.3185     30.71183333 16.03516667 35.69933333
  47.01       39.34116667 19.16766667]
 [23.43433333  0.         16.915      13.746      20.15466667 14.1895
  22.533      22.7555     25.69083333]
 [11.00133333 16.6485      0.         22.84233333  8.392      26.64433333
  37.955      33.67216667 16.45033333]
 [28.49583333 13.86733333 21.97633333  0.         26.79366667 24.81916667
  24.38733333 10.48333333 34

In [ ]:
# Example: weight in kg for each stop (same order as locations)
weights = [0, 200, 150, 100, 200, 300, 250, 150, 250]  # 0 for warehouse

In [ ]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# Define vehicle capacity (e.g., 2000 kg)
vehicle_capacity = 2000

# OR-Tools setup
manager = pywrapcp.RoutingIndexManager(len(distance_matrix), 1, 0)
routing = pywrapcp.RoutingModel(manager)

def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(distance_matrix[from_node][to_node] * 1000)  # meters

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add capacity constraint
def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return weights[from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    [vehicle_capacity],  # vehicle maximum capacity
    True,  # start cumul to zero
    'Capacity'
)

# 5. Solve
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
solution = routing.SolveWithParameters(search_parameters)

# 6. Extract and print the route
if solution:
    index = routing.Start(0)
    route = []
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        route.append(node)
        index = solution.Value(routing.NextVar(index))
    route.append(manager.IndexToNode(index))
    print("Optimal route (indices):", route)
    print("Optimal route (addresses):")
    for idx in route:
        print(f"{idx+1}. {addresses[idx]}")
    # Calculate total distance
    total_distance = 0
    for i in range(len(route)-1):
        total_distance += distance_matrix[route[i]][route[i+1]]
    print(f"\nTotal distance: {total_distance:.2f} km")
else:
    print("No solution found!")

# 7. Plot on Folium map with reduced arrow frequency
romania_center = [44.4268, 26.1025]
m = folium.Map(location=romania_center, zoom_start=11)

# Add markers
for i, idx in enumerate(route):
    lat, lon = locations[idx][1], locations[idx][0]
    folium.Marker(
        [lat, lon],
        popup=f"{i+1}. {addresses[idx]} (Weight: {weights[idx]} kg)",
        tooltip=f"Stop {i+1}",
        icon=folium.Icon(color='blue' if i not in [0, len(route)-1] else 'red')
    ).add_to(m)

# Draw route with spaced arrows
route_coords = [[locations[idx][1], locations[idx][0]] for idx in route]
polyline = folium.PolyLine(route_coords, color="green", weight=2.5, opacity=1)
polyline.add_to(m)
arrows = PolyLineTextPath(
    polyline,
    '                    ➔                    ',  # lots of spaces for low frequency
    repeat=True,
    offset=7,
    attributes={'fill': 'green', 'font-weight': 'bold', 'font-size': '18'}
)
arrows.add_to(m)

Optimal route (indices): [0, 2, 4, 8, 1, 5, 7, 3, 6, 0]
Optimal route (addresses):
1. Strada Preciziei 24, Chiajna, Ilfov, Romania
3. Bulevardul Iuliu Maniu 220, Bucharest, Romania
5. Strada Drumul Taberei 34, Bucharest, Romania
9. Prelungirea Ghencea 85, Domnești, Ilfov, Romania
2. Piața Unirii, Bucharest, Romania
6. Șoseaua Olteniței 103, Popești-Leordeni, Ilfov, Romania
8. Strada Erou Iancu Nicolae 67, Voluntari, Ilfov, Romania
4. Strada Barbu Văcărescu 201, Bucharest, Romania
7. Strada 1 Decembrie 1918 12, Pantelimon, Ilfov, Romania
1. Strada Preciziei 24, Chiajna, Ilfov, Romania

Total distance: 94.82 km


In [ ]:
m

## **1: Parameters, Input Data, and Setup**

In [ ]:
# Install required packages
!pip install openrouteservice ortools pandas folium

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# 1) Prepare
addresses2 = [
    "Preciziei 24, Chiajna, Ilfov, Romania",   # depot
    "Piața Unirii, Bucharest, Romania",
    "Bulevardul Iuliu Maniu 220, Bucharest, Romania",
    "Strada Barbu Văcărescu 201, Bucharest, Romania",
    "Strada Drumul Taberei 34, Bucharest, Romania",
    "Șoseaua Olteniței 103, Popești-Leordeni, Ilfov, Romania",
    "Strada 1 Decembrie 1918 12, Pantelimon, Ilfov, Romania",
    "Strada Erou Iancu Nicolae 67, Voluntari, Ilfov, Romania",
    "Prelungirea Ghencea 85, Domnești, Ilfov, Romania",
    "Bulevardul Timișoara 50, Bucharest, Romania",
    "Aleea Petricani 14, Bucharest, Romania",
    "Strada Someșului 22, Bucharest, Romania",
    "Șoseaua Pantelimon 302, Bucharest, Romania",
    "Calea Moșilor 150, Bucharest, Romania",
    "Bulevardul Lacul Tei 10, Bucharest, Romania"
]

# 2) Create a geolocator with a rate‐limiter (to respect OSM’s usage policy)
geolocator = Nominatim(user_agent="logistics-demo")
geocode    = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# 3) Geocode all addresses
coords = []
for addr in addresses2:
    location = geocode(addr)
    if location:
        coords.append((location.longitude, location.latitude))
    else:
        coords.append((None, None))  # failed to geocode

print(coords)

coords = [
    (25.9696, 44.4481),
    (26.1025, 44.4268),
    (26.0122, 44.4352),
    (26.1097, 44.4781),
    (25.9987, 44.4260),
    (26.1457, 44.3755),
    (26.2032, 44.4547),
    (26.1427, 44.5076),
    (25.9692, 44.3972)
]
demands_tons = [0, 2, 5, -2, 4, -1, 3, 1, -0.5]

[(None, None), (26.1023154, 44.4279146), (26.0058172, 44.4348733), (26.1024352, 44.4792527), (None, None), (26.1502545, 44.3819727), (None, None), (26.1264037, 44.5138947), (None, None), (26.0213671, 44.4266816), (None, None), (26.0611254, 44.4617082), (26.1312654, 44.4486338), (26.1127257, 44.4369416), (26.1061996, 44.4570675)]


In [ ]:
import openrouteservice
import pandas as pd
import numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import re
from google.colab import userdata

# 1. PARAMETERS

fleet = [
    {
        "type": "7.5t",
        "capacity": 7500,           # kg
        "fixed_cost": 100,          # RON/day
        "variable_cost_km": 1.5,    # RON/km
        "variable_cost_ton_km": 0.5 # RON/ton-km
    },
    {
        "type": "12t",
        "capacity": 12000,          # kg
        "fixed_cost": 130,          # RON/day
        "variable_cost_km": 1.7,    # RON/km
        "variable_cost_ton_km": 0.6 # RON/ton-km
    }
]

revenue_per_ton_km = 2.5      # RON/ton-km
service_time_per_ton = 10      # minutes per ton
traffic_margin = 1.2          # 20% extra for traffic
working_window_min = 480      # 8 hours in minutes

# 2. LOCATIONS & DEMANDS
addresses = [
    "Preciziei 24, Chiajna, Ilfov, Romania",   # depot
    "Piața Unirii, Bucharest, Romania",
    "Bulevardul Iuliu Maniu 220, Bucharest, Romania",
    "Strada Barbu Văcărescu 201, Bucharest, Romania",
    "Strada Drumul Taberei 34, Bucharest, Romania",
    "Șoseaua Olteniței 103, Popești-Leordeni, Ilfov, Romania",
    "Strada 1 Decembrie 1918 12, Pantelimon, Ilfov, Romania",
    "Strada Erou Iancu Nicolae 67, Voluntari, Ilfov, Romania",
    "Prelungirea Ghencea 85, Domnești, Ilfov, Romania",
    "Bulevardul Timișoara 50, Bucharest, Romania",
    "Aleea Petricani 14, Bucharest, Romania",
    "Strada Someșului 22, Bucharest, Romania",
    "Șoseaua Pantelimon 302, Bucharest, Romania",
    "Calea Moșilor 150, Bucharest, Romania",
    "Bulevardul Lacul Tei 10, Bucharest, Romania"
]

coords = [
    (25.9696, 44.4481),
    (26.1025, 44.4268),
    (26.0122, 44.4352),
    (26.1097, 44.4781),
    (25.9987, 44.4260),
    (26.1457, 44.3755),
    (26.2032, 44.4547),
    (26.1427, 44.5076),
    (25.9692, 44.3972),
    # new coords:
    (26.0050, 44.4200),
    (26.1080, 44.4900),
    (26.0650, 44.4380),
    (26.2130, 44.4420),
    (26.1510, 44.4300),
    (26.1600, 44.4590)
]
demands_tons = [0, 2, 5, -2, 4, -1, 3, 1, -0.5, 6, -3, 4.5, -2, 2.5, 5]

# regex to remove anything that isn't A–Z, a–z or 0–9
clean = lambda s: re.sub(r'[^\w]', '', s, flags=re.UNICODE).replace('_', '')

locations = [
    {
        "name": " ".join(clean(w) for w in addr.split()[1:3]),
        "coords": coord,
        "demand_ton": d
    }
    for addr, coord, d in zip(addresses, coords, demands_tons)
]

# 3. ORS CLIENT
ORS_API_KEY = userdata.get("ORS_API_KEY")
client = openrouteservice.Client(key=ORS_API_KEY)

coords_list = [loc["coords"] for loc in locations]
print("Fetching matrix...")
matrix = client.distance_matrix(
    locations=coords_list,
    profile='driving-hgv',
    metrics=['distance', 'duration'],
    units='m'
)
distances_km = np.array(matrix['distances']) / 1000
durations_min = np.array(matrix['durations']) * traffic_margin / 60

# 4. SERVICE TIMES

service_times = [abs(d)*service_time_per_ton for d in demands_tons]

# 5. BUILD TIME MATRIX

num_locations = len(locations)
time_matrix = []
for i in range(num_locations):
    row = []
    for j in range(num_locations):
        if i == j:
            row.append(0)
        else:
            row.append(durations_min[i][j] + service_times[j])
    time_matrix.append(row)

# 6. FLEET SETUP

fleet_plan = [
    {"type": "7.5t", "count": 3},
    {"type": "12t", "count": 2}
]

vehicle_caps       = []
vehicle_fixed      = []
vehicle_var_km     = []
vehicle_var_tonkm  = []
vehicle_types      = []

for plan in fleet_plan:
    specs = next(f for f in fleet if f["type"] == plan["type"])
    for _ in range(plan["count"]):
        vehicle_caps.append(specs["capacity"])
        vehicle_fixed.append(specs["fixed_cost"])
        vehicle_var_km.append(specs["variable_cost_km"])
        vehicle_var_tonkm.append(specs["variable_cost_ton_km"])
        vehicle_types.append(specs["type"])

num_vehicles = len(vehicle_caps)
depot_index  = 0

# 7. PRE-CALCULATE DEPOT→CUSTOMER REVENUES

# max possible revenue for C constant
max_demand = max(abs(d) for d in demands_tons)
max_dist   = distances_km[depot_index].max()
C = revenue_per_ton_km * max_demand * max_dist

dep2cust_revenue = []
for i, loc in enumerate(locations):
    if i == depot_index:
        dep2cust_revenue.append(0)
    else:
        r = revenue_per_ton_km * abs(loc["demand_ton"]) * distances_km[depot_index][i]
        dep2cust_revenue.append(r)

# 8. OR-TOOLS SETUP

manager = pywrapcp.RoutingIndexManager(num_locations, num_vehicles, depot_index)
routing = pywrapcp.RoutingModel(manager)

# DEMAND / CAPACITY

demands_kg = [int(loc["demand_ton"]*1000) for loc in locations]
def demand_cb(idx):
    node = manager.IndexToNode(idx)
    return abs(demands_kg[node])

d_cb_idx = routing.RegisterUnaryTransitCallback(demand_cb)
routing.AddDimensionWithVehicleCapacity(
    d_cb_idx, 0, vehicle_caps, True, 'Capacity')

# TIME

def time_cb(i, j):
    ni = manager.IndexToNode(i)
    nj = manager.IndexToNode(j)
    return int(time_matrix[ni][nj]*100)
t_cb_idx = routing.RegisterTransitCallback(time_cb)
routing.AddDimension(
    t_cb_idx, 0, working_window_min*100, True, 'Time'
)

# DISJUNCTIONS

penalty = 500000  # scaled down
for node in range(1, num_locations):
    routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

# FIXED COSTS

for v in range(num_vehicles):
    high_fixed = vehicle_fixed[v] * 10
    routing.SetFixedCostOfVehicle(int(high_fixed * 100), v)

# ARC COST (shifted)

def make_profit_cb(v):
    def cb(i, j):
        ni = manager.IndexToNode(i)
        nj = manager.IndexToNode(j)
        # revenue always depot→nj
        rev = dep2cust_revenue[nj]
        # true cost from ni→nj
        dkm = distances_km[ni][nj]
        tons = abs(locations[nj]["demand_ton"])
        cost_km    = vehicle_var_km[v]    * dkm
        cost_tonkm = vehicle_var_tonkm[v] * tons * dkm
        cost = cost_km + cost_tonkm
        raw = cost - rev
        arc = raw + C
        return int(arc * 100)
    return cb

for v in range(num_vehicles):
    cb_idx = routing.RegisterTransitCallback(make_profit_cb(v))
    routing.SetArcCostEvaluatorOfVehicle(cb_idx, v)

# 9. SOLVE

params = pywrapcp.DefaultRoutingSearchParameters()
params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
params.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
params.time_limit.seconds = 60

sol = routing.SolveWithParameters(params)

# 10. PRINT SOLUTION

if not sol:
    print("No solution!")
else:
    print(f"Objective: {sol.ObjectiveValue()/100:.2f} (includes C-offset)")

    for v in range(num_vehicles):
        idx = routing.Start(v)
        if routing.IsEnd(sol.Value(routing.NextVar(idx))):
            continue
        print(f"\nRoute for vehicle {v} ({vehicle_types[v]}):")
        total_dist = 0
        total_rev  = 0
        total_cost = vehicle_fixed[v]
        while not routing.IsEnd(idx):
            ni = manager.IndexToNode(idx)
            next_idx = sol.Value(routing.NextVar(idx))
            nj = manager.IndexToNode(next_idx)

            if nj != depot_index:
                # accumulate revenue
                total_rev += dep2cust_revenue[nj]
            dkm = distances_km[ni][nj]
            tons = abs(locations[nj]["demand_ton"])
            total_cost += vehicle_var_km[v]*dkm + vehicle_var_tonkm[v]*tons*dkm
            total_dist += dkm
            idx = next_idx

        profit = total_rev - total_cost
        print(f"  Distance: {total_dist:.1f} km")
        print(f"  Revenue:  {total_rev:.2f} RON")
        print(f"  Cost:     {total_cost:.2f} RON")
        print(f"  Profit:   {profit:.2f} RON")


Fetching matrix...
Objective: 15207.06 (includes C-offset)

Route for vehicle 0 (7.5t):
  Distance: 39.6 km
  Revenue:  203.75 RON
  Cost:     187.33 RON
  Profit:   16.42 RON

Route for vehicle 1 (7.5t):
  Distance: 11.3 km
  Revenue:  65.47 RON
  Cost:     129.98 RON
  Profit:   -64.51 RON

Route for vehicle 2 (7.5t):
  Distance: 61.9 km
  Revenue:  359.32 RON
  Cost:     224.27 RON
  Profit:   135.05 RON

Route for vehicle 3 (12t):
  Distance: 56.0 km
  Revenue:  722.18 RON
  Cost:     276.38 RON
  Profit:   445.79 RON

Route for vehicle 4 (12t):
  Distance: 15.3 km
  Revenue:  162.02 RON
  Cost:     177.26 RON
  Profit:   -15.24 RON


In [ ]:
# --- PRINT DROPPED LOCATIONS ---

dropped = []
# skip index 0 (the depot), check each customer node
for node in range(1, len(locations)):
    idx = manager.NodeToIndex(node)
    # if NextVar(idx) == idx, that node was never left → it was dropped
    if sol.Value(routing.NextVar(idx)) == idx:
        dropped.append((node, locations[node]['name'], demands_tons[node]))

if not dropped:
    print("👍 All customer locations were served.")
else:
    print("❌ Dropped locations:")
    for node, name, demand in dropped:
        print(f" • Node {node}: {name} (demand = {demand} t)")


👍 All customer locations were served.


In [ ]:
import pandas as pd

# --- Route Leg Details with Direct Cost Calculations ---

rows = []
num_vehicles = len(vehicle_caps)

for v in range(num_vehicles):
    # 1) Collect all non-zero customer stops for this truck
    stops = []
    idx = routing.Start(v)
    while not routing.IsEnd(idx):
        nxt  = sol.Value(routing.NextVar(idx))
        node = manager.IndexToNode(nxt)
        if node != depot_index and demands_tons[node] != 0:
            stops.append(node)
        idx = nxt

    # Skip empty routes
    if not stops:
        continue

    # 2) Compute initial onboard load (sum of deliveries only)
    load_onboard = sum(demands_tons[n] * 1000 for n in stops if demands_tons[n] > 0)
    if load_onboard > vehicle_caps[v]:
        print(f"⚠️ Truck {v+1}: initial load {load_onboard} kg exceeds capacity {vehicle_caps[v]} kg")

    # 3) Walk each leg, computing costs directly
    idx = routing.Start(v)
    prev_node = manager.IndexToNode(idx)
    while not routing.IsEnd(idx):
        nxt       = sol.Value(routing.NextVar(idx))
        node      = manager.IndexToNode(nxt)

        # Distances & times
        dist_km   = distances_km[prev_node][node]
        time_leg  = durations_min[prev_node][node]
        service   = service_times[node] if node != depot_index else 0

        # Demand at this stop (tons)
        tons = abs(locations[node]["demand_ton"])
        demand_t     = locations[prev_node]["demand_ton"]
        quantity_kg  = int(demand_t * 1000)

        # ---- Direct cost formulas ----
        base_cost  = vehicle_var_km[v]    * dist_km
        load_cost  = vehicle_var_tonkm[v] * tons * dist_km
        total_cost = base_cost + load_cost

        # Revenue and profit
        revenue_leg = revenue_per_ton_km * tons * distances_km[depot_index][node]
        profit_leg  = revenue_leg - total_cost

        # Load % and deadhead
        pct_load = (load_onboard / vehicle_caps[v] * 100)
        deadhead = (load_onboard == 0)

        # CO₂ footprint
        co2 = 0.27 * dist_km

        # Record the leg
        rows.append({
            "Truck":            v+1,
            "Leg":              f"{locations[prev_node]['name']} → {locations[node]['name']}",
            "Km":               round(dist_km, 2),
            "Qty (kg)":         quantity_kg,
            "Load (kg)":        load_onboard,
            "% Load":           f"{pct_load:.1f}%",
            "Time/leg (min)":   round(time_leg, 1),
            "Service (min)":    service,
            "Base Cost (RON)":  round(base_cost, 2),
            "Load Cost (RON)":  round(load_cost, 2),
            "Total Cost (RON)": round(total_cost, 2),
            "Revenue (RON)":    round(revenue_leg, 2),
            "Profit/leg (RON)": round(profit_leg, 2),
            "CO₂ (kg)":         round(co2, 2),
            "Deadhead?":        "Yes" if deadhead else "No"
        })

        # Update onboard load (deliveries subtract, pickups add)
        load_onboard -= demands_tons[node] * 1000

        prev_node = node
        idx       = nxt

# 4) Build DataFrame and display
df_legs = pd.DataFrame(rows)
print("\n--- Route Leg Details (Corrected Costs) ---")
display(df_legs)

cap_map = {i+1: cap for i, cap in enumerate(vehicle_caps)}
df_legs["Avg Load (%)"] = df_legs.apply(
    lambda r: r["Load (kg)"] / cap_map[r["Truck"]] * 100, axis=1)

# --- Subtotals per truck with averages ---
totals = df_legs.groupby("Truck", as_index=False).agg({
    "Km":               "sum",
    "Avg Load (%)":     "mean",
    "Time/leg (min)":   "sum",
    "Service (min)":    "sum",
    "Base Cost (RON)":  "sum",
    "Load Cost (RON)":  "sum",
    "Total Cost (RON)": "sum",
    "Revenue (RON)":    "sum",
    "Profit/leg (RON)": "sum",
    "CO₂ (kg)":         "sum"
})

# Add fixed cost and net profit
totals["Fixed Cost (RON)"] = [vehicle_fixed[t-1] for t in totals["Truck"]]
totals["Net Profit (RON)"] = totals["Profit/leg (RON)"] - totals["Fixed Cost (RON)"]
totals["Avg Load (%)"]  = totals["Avg Load (%)"].map(lambda x: f"{x:.1f}%")

# Reorder so Avg Load and time/service follow Km, CO₂ last
cols = [
    "Truck", "Km", "Avg Load (%)", "Time/leg (min)", "Service (min)",
    "Base Cost (RON)", "Load Cost (RON)", "Total Cost (RON)",
    "Revenue (RON)", "Profit/leg (RON)", "Fixed Cost (RON)", "Net Profit (RON)",
    "CO₂ (kg)"
]
totals = totals[cols]

print("\n--- Subtotals per Truck ---")
display(totals)

# 6) General total
general = pd.DataFrame([{
    "Truck":            "All",
    "Km":               totals["Km"].sum(),
    "Avg Load (%)":    df_legs["Avg Load (%)"].mean(),
    "Time/leg (min)":   df_legs["Time/leg (min)"].sum(),
    "Service (min)":    df_legs["Service (min)"].sum(),
    "Base Cost (RON)":  totals["Base Cost (RON)"].sum(),
    "Load Cost (RON)":  totals["Load Cost (RON)"].sum(),
    "Total Cost (RON)": totals["Total Cost (RON)"].sum(),
    "Revenue (RON)":    totals["Revenue (RON)"].sum(),
    "Profit/leg (RON)": totals["Profit/leg (RON)"].sum(),
    "Fixed Cost (RON)": totals["Fixed Cost (RON)"].sum(),
    "Net Profit (RON)": totals["Net Profit (RON)"].sum(),
    "CO₂ (kg)":         totals["CO₂ (kg)"].sum()
}])

# Apply same column order
general = general[cols]
general["Avg Load (%)"] = general["Avg Load (%)"].map(lambda x: f"{x:.1f}%")

print("\n--- General Total ---")
display(general)


--- Route Leg Details (Corrected Costs) ---


,Truck,Leg,Km,Qty (kg),Load (kg),% Load,Time/leg (min),Service (min),Base Cost (RON),Load Cost (RON),Total Cost (RON),Revenue (RON),Profit/leg (RON),CO₂ (kg),Deadhead?
0,1,24 Chiajna → Ghencea 85,9.69,0,6500.0,86.7%,23.0,5.0,14.53,2.42,16.96,12.11,-4.84,2.62,No
1,1,Ghencea 85 → Unirii Bucharest,13.67,-500,7000.0,93.3%,30.2,20.0,20.51,13.67,34.19,71.99,37.80,3.69,No
2,1,Unirii Bucharest → Someșului 22,5.28,2000,5000.0,66.7%,11.5,45.0,7.92,11.88,19.79,119.65,99.86,1.43,No
3,1,Someșului 22 → 24 Chiajna,10.93,4500,500.0,6.7%,23.5,0.0,16.39,0.00,16.39,0.00,-16.39,2.95,No
4,2,24 Chiajna → Iuliu Maniu,5.24,0,5000.0,66.7%,13.6,50.0,7.86,13.09,20.95,65.47,44.52,1.41,No
5,2,Iuliu Maniu → 24 Chiajna,6.02,5000,0.0,0.0%,13.2,0.0,9.03,0.00,9.03,0.00,-9.03,1.63,Yes
6,3,24 Chiajna → Olteniței 103,20.30,0,3500.0,46.7%,42.8,10.0,30.45,10.15,40.60,50.75,10.15,5.48,No
7,3,Olteniței 103 → Moșilor 150,7.32,-1000,4500.0,60.0%,17.3,25.0,10.98,9.15,20.13,117.30,97.17,1.98,No
8,3,Moșilor 150 → Erou Iancu,11.54,2500,2000.0,26.7%,26.6,10.0,17.31,5.77,23.08,60.89,37.81,3.12,No
9,3,Erou Iancu → Petricani 14,4.21,1000,1000.0,13.3%,13.3,30.0,6.32,6.32,12.63,130.37,117.74,1.14,No



--- Subtotals per Truck ---


,Truck,Km,Avg Load (%),Time/leg (min),Service (min),Base Cost (RON),Load Cost (RON),Total Cost (RON),Revenue (RON),Profit/leg (RON),Fixed Cost (RON),Net Profit (RON),CO₂ (kg)
0,1,39.57,63.3%,88.2,70.0,59.35,27.97,87.33,203.75,116.43,100,16.43,10.69
1,2,11.26,33.3%,26.8,50.0,16.89,13.09,29.98,65.47,35.49,100,-64.51,3.04
2,3,61.91,40.0%,141.1,75.0,92.88,31.39,124.26,359.31,235.05,100,135.05,16.73
3,4,55.96,56.7%,134.4,120.0,95.12,51.27,146.39,722.18,575.79,130,445.79,15.11
4,5,15.31,44.4%,47.8,100.0,26.02,21.23,47.26,162.02,114.77,130,-15.23,4.13



--- General Total ---


,Truck,Km,Avg Load (%),Time/leg (min),Service (min),Base Cost (RON),Load Cost (RON),Total Cost (RON),Revenue (RON),Profit/leg (RON),Fixed Cost (RON),Net Profit (RON),CO₂ (kg)
0,All,184.01,49.3%,438.3,415.0,290.26,144.95,435.22,1512.73,1077.53,560,517.53,49.7


In [ ]:
from folium import Map, Marker, PolyLine, Icon
from folium.plugins import PolyLineTextPath

# Center initially on Romania
romania_center = [44.4268, 26.1025]
m = Map(location=romania_center, zoom_start=11)

color_map = {1: "blue", 2: "red", 3: "green", 4: "purple", 5: "orange"}

all_points = []  # collect all lat/lon to compute bounds at the end
num_vehicles = manager.GetNumberOfVehicles()

for v in range(num_vehicles):
    truck_id = v + 1
    if truck_id not in df_legs["Truck"].unique():
        continue

    # Reconstruct this truck's route (including depot at start & end)
    route = []
    idx = routing.Start(v)
    while not routing.IsEnd(idx):
        node = manager.IndexToNode(idx)
        route.append(node)
        idx = sol.Value(routing.NextVar(idx))
    route.append(manager.IndexToNode(idx))

    # Plot each stop
    for i, node in enumerate(route):
        lat, lon = locations[node]["coords"][1], locations[node]["coords"][0]
        all_points.append((lat, lon))
        weight_kg = abs(demands_tons[node]) * 1000
        is_endpoint = (i == 0 or i == len(route)-1)
        Marker(
            [lat, lon],
            popup=f"{i+1}. {locations[node]['name']} (Weight: {weight_kg:.0f} kg)",
            tooltip=f"Stop {i+1}",
            icon=Icon(color="red" if is_endpoint else "blue")
        ).add_to(m)

    # Draw the polyline
    coords = [[lat, lon] for (lat, lon) in all_points[-len(route):]]
    line = PolyLine(
        locations=coords,
        color=color_map.get(truck_id, "black"),
        weight=3,
        opacity=0.8,
        tooltip=f"Truck {truck_id} route"
    ).add_to(m)

    # Overlay very sparse arrows
    PolyLineTextPath(
        line,
        ' ' * 20 + '➔' + ' ' * 20,
        repeat=True,
        offset=7,
        attributes={
            'fill': color_map.get(truck_id, "black"),
            'font-weight': 'bold',
            'font-size': '18'
        }
    ).add_to(m)

# Fit the map to show all points
if all_points:
    lats, lons = zip(*all_points)
    m.fit_bounds([[min(lats), min(lons)], [max(lats), max(lons)]])

# Finally display
m


## **Gradio Demo**

In [ ]:
!pip install openrouteservice ortools pandas folium

In [69]:
# ─────────────────────── ①  CORE & SOLVER  ───────────────────────
import json, io, tempfile, datetime, warnings, textwrap, os, re
import pandas as pd, numpy as np, openrouteservice, folium
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
from folium import Map, Marker, PolyLine, Icon
from folium.plugins import PolyLineTextPath
from google.colab import userdata
warnings.filterwarnings("ignore", category=UserWarning)

# ---------- A.  VRP solver ----------
def solve_vrp(params: dict, df_loc: pd.DataFrame):
    # -------- 1. unpack ----------
    fleet              = params["fleet"]
    rev_ton_km         = params["revenue_per_ton_km"]
    svc_per_ton        = params["service_time_per_ton"]
    traffic_margin     = params["traffic_margin"]
    working_window_min = params["working_window_min"]

    # get ors api key
    key = params.get("ors_api_key", "").strip()

    # treat “PASTE_KEY”, empty string, or missing field as “no key”
    if not key or key.upper() == "PASTE_KEY":
        # try env var
        key = os.getenv("ORS_API_KEY", "").strip()
    # try Colab > Secrets last
    if (not key) and userdata:
        key = (userdata.get("ORS_API_KEY") or "").strip()

    if not key:
        raise ValueError(
            "OpenRouteService API key missing – add it in the Parameters JSON "
            "or store ORS_API_KEY as an environment / Colab secret."
        )

    ors_key = key

    # -------- 2. locations ----------
    locs   = df_loc.to_dict("records")
    coords = [(r["lon"], r["lat"]) for r in locs]
    demands = [r["demand_ton"] for r in locs]
    depot   = 0

    # -------- 3. ORS matrix ----------
    client  = openrouteservice.Client(key=ors_key)
    mtx     = client.distance_matrix(coords, profile="driving-hgv",
                                     metrics=["distance", "duration"], units="m")
    dist_km = np.array(mtx["distances"]) / 1000
    dur_min = np.array(mtx["durations"]) * traffic_margin / 60
    svc     = [abs(d) * svc_per_ton for d in demands]
    time_mx = (dur_min + np.array(svc)).tolist()
    for i in range(len(time_mx)):
        time_mx[i][i] = 0

    # -------- 4. expand fleet ----------
    caps, fixed, c_km, c_tkm, types = [], [], [], [], []
    for spec in fleet:
        for _ in range(spec["count"]):
            caps.append(spec["capacity"])
            fixed.append(spec["fixed_cost"])
            c_km.append(spec["variable_cost_km"])
            c_tkm.append(spec["variable_cost_ton_km"])
            types.append(spec["type"])
    V = len(caps)

    # -------- 5. OR-Tools ----------
    mgr = pywrapcp.RoutingIndexManager(len(locs), V, depot)
    rt  = pywrapcp.RoutingModel(mgr)

    # demand dimension
    dkg = [int(abs(d) * 1000) for d in demands]
    rt.AddDimensionWithVehicleCapacity(
        rt.RegisterUnaryTransitCallback(lambda i: dkg[mgr.IndexToNode(i)]),
        0, caps, True, "Cap"
    )

    # time dimension
    rt.AddDimension(
        rt.RegisterTransitCallback(
            lambda i, j: int(time_mx[mgr.IndexToNode(i)][mgr.IndexToNode(j)] * 100)
        ),
        0, working_window_min * 100, True, "Time"
    )

    # arc costs (profit-shift)
    C = rev_ton_km * max(abs(np.array(demands))) * dist_km[depot].max()
    dep2rev = [0] + [
        rev_ton_km * abs(d) * dist_km[depot][i] for i, d in enumerate(demands) if i > 0
    ]

    for v in range(V):
        def make(vh):
            def arc(i, j):
                ni, nj = mgr.IndexToNode(i), mgr.IndexToNode(j)
                tons   = abs(demands[nj])
                cost   = c_km[vh] * dist_km[ni][nj] + c_tkm[vh] * tons * dist_km[ni][nj]
                return int((cost - dep2rev[nj] + C) * 100)
            return arc
        rt.SetArcCostEvaluatorOfVehicle(rt.RegisterTransitCallback(make(v)), v)
        rt.SetFixedCostOfVehicle(int(fixed[v] * 100 * 10), v)

    for n in range(1, len(locs)):
        rt.AddDisjunction([mgr.NodeToIndex(n)], 5_00_000)

    # solve
    sp = pywrapcp.DefaultRoutingSearchParameters()
    sp.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    sp.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    sp.time_limit.seconds = 20
    sol = rt.SolveWithParameters(sp)
    if not sol:
        raise RuntimeError("No solution in 20 s – relax constraints.")

    # ---------- 6. detailed leg list ------------------------------------------------
    def short(addr):                       # keep leg text compact
        return addr.split(",")[0].strip()

    legs, dropped = [], []
    for n in range(1, len(locs)):
        if sol.Value(rt.NextVar(mgr.NodeToIndex(n))) == mgr.NodeToIndex(n):
            dropped.append(n)

    for v in range(V):
        if rt.IsEnd(sol.Value(rt.NextVar(rt.Start(v)))):
            continue                                    # unused vehicle

        # customers on this route (exclude depot)
        idx = rt.Start(v)
        cust_nodes = []
        while not rt.IsEnd(idx):
            idx  = sol.Value(rt.NextVar(idx))
            node = mgr.IndexToNode(idx)
            if node != depot:
                cust_nodes.append(node)

        load_onboard = sum(max(0, demands[n]) * 1000 for n in cust_nodes)

        idx = rt.Start(v)
        while not rt.IsEnd(idx):
            nxt  = sol.Value(rt.NextVar(idx))
            ni, nj = mgr.IndexToNode(idx), mgr.IndexToNode(nxt)

            dist   = dist_km[ni][nj]
            base_c = c_km[v]  * dist
            load_c = c_tkm[v] * abs(demands[nj]) * dist
            tot_c  = base_c + load_c
            rev    = rev_ton_km * abs(demands[nj]) * dist_km[depot][nj]
            profit = rev - tot_c
            pct_ld = min(load_onboard, caps[v]) / caps[v] * 100  # clamp ≤100 %

            legs.append({
                "Truck": v + 1,
                "From":  short(locs[ni]["name"]),
                "To":    short(locs[nj]["name"]),
                "Km":          round(dist, 2),
                "Qty (kg)":    int(demands[nj] * 1000),          # keep sign
                "Load (kg)":   load_onboard,
                "% Load":      f"{pct_ld:.1f}%",
                "Time (min)":  round(dur_min[ni][nj], 1),
                "Service (min)": svc[nj] if nj != depot else 0,
                "Base RON":    round(base_c,  2),
                "Load RON":    round(load_c,  2),
                "Total RON":   round(tot_c,   2),
                "Revenue RON": round(rev,     2),
                "Profit RON":  round(profit,  2),
                "CO2 (kg)":    round(0.27 * dist, 2),
                "Deadhead?":   "Yes" if load_onboard == 0 else "No"
            })
            load_onboard -= demands[nj] * 1000
            idx = nxt

    df_legs = pd.DataFrame(legs)

    # ---------- 7. subtotals ---------------------------------------------------------
    cap_map     = {i + 1: caps[i] for i in range(V)}
    load_pct    = df_legs["Load (kg)"] / df_legs["Truck"].map(cap_map) * 100
    subtot_cols = {
        "Km":               "sum",
        "Time (min)":       "sum",
        "Service (min)":    "sum",
        "Base RON":         "sum",
        "Load RON":         "sum",
        "Total RON":        "sum",
        "Revenue RON":      "sum",
        "Profit RON":       "sum",
        "CO2 (kg)":         "sum"
    }

    subtot = (
        df_legs.assign(**{"Avg Load (%)": load_pct})
              .groupby("Truck", as_index=False)
              .agg({**subtot_cols, "Avg Load (%)": "mean"})
    )
    subtot["Avg Load (%)"] = subtot["Avg Load (%)"].map(lambda x: f"{x:.1f}%")
    subtot["Fixed RON"]    = [fixed[t-1] for t in subtot["Truck"]]
    subtot["Net RON"]      = subtot["Profit RON"] - subtot["Fixed RON"]

    summary_cols = ["Truck", "Km", "Avg Load (%)", "Time (min)", "Service (min)",
                    "Base RON", "Load RON", "Total RON", "Revenue RON",
                    "Profit RON", "Fixed RON", "Net RON", "CO2 (kg)"]

    tot_row = pd.DataFrame([{
        "Truck":        "All",
        **{c: subtot[c].sum() for c in summary_cols if c not in ("Truck", "Avg Load (%)")},
        "Avg Load (%)": f"{load_pct.mean():.1f}%"
    }])

    # styled HTML (yellow header **and** total row)
    style = """
    <style>
    .custom  {border-collapse:collapse; font-size:13px;}
    .custom th {background:#ffe599; font-weight:bold; padding:4px;
              text-align:center;}
    .custom td {padding:4px; text-align:right;}
    .custom tr:last-child td {background:#ffe599; font-weight:bold;}
    </style>
    """
    totals_html = style + pd.concat([subtot[summary_cols], tot_row[summary_cols]],
                                    ignore_index=True).to_html(index=False,
                                                                classes="custom",
                                                                border=0)

    # ---------- 8. coloured Folium map (arrows, legend) -----------------------------
    from folium.plugins import PolyLineTextPath
    m = folium.Map(location=[df_loc.lat.mean(), df_loc.lon.mean()], zoom_start=11)
    color_map = {1:"blue", 2:"red", 3:"green", 4:"purple", 5:"orange", 6:"black"}
    all_pts = []

    for v in range(V):
        tid = v + 1
        if tid not in df_legs["Truck"].unique():
            continue
        idx = rt.Start(v); nodes=[]
        while not rt.IsEnd(idx):
            nodes.append(mgr.IndexToNode(idx))
            idx = sol.Value(rt.NextVar(idx))
        nodes.append(mgr.IndexToNode(idx))

        pts = [(locs[n]["lat"], locs[n]["lon"]) for n in nodes]
        all_pts.extend(pts)

        line = folium.PolyLine(
            pts, color=color_map.get(tid,"black"), weight=2, opacity=0.8,
            tooltip=f"Truck {tid}"
        ).add_to(m)
        arrow_pattern = " " * 15 + "➔" + " " * 15      # long blanks ⇒ sparse arrows
        PolyLineTextPath(
            line,
            arrow_pattern,
            repeat=True,
            offset=7,
            attributes={
                "fill":  color_map.get(tid, "black"),
                "font-weight": "bold",
                "font-size":  "18"
            }
        ).add_to(m)

        for i, (la, lo) in enumerate(pts):
            node   = nodes[i]                       # node id on this leg
            demand = demands[node]                  # tonnes (‐ for pickup)
            label  = f"{i+1}. {short(locs[node]['name'])} (Demand: {demand} t)"

            folium.Marker(
                [la, lo],
                tooltip=label,                      # shows on hover
                popup=label,                        # shows on click
                icon=folium.Icon(
                    color="red" if i in (0, len(pts)-1) else "blue"
                )
            ).add_to(m)

    if all_pts:
        m.fit_bounds([[min(p[0] for p in all_pts), min(p[1] for p in all_pts)],
                      [max(p[0] for p in all_pts), max(p[1] for p in all_pts)]])

    legend = ('<div style="position:fixed;bottom:15px;left:15px;z-index:9999;'
              'background:white;border:1px solid #999;padding:6px">'
              '<b>Truck colours</b><br>' +
              ''.join(f'<i style="background:{c};width:12px;height:12px;display:inline-block"></i>'
                      f' Truck {tid}<br>'
                      for tid,c in color_map.items() if tid<=V) +
              '</div>')
    m.get_root().html.add_child(folium.Element(legend))

    map_html = (
        '<iframe srcdoc="' +
        m.get_root().render().replace('"','&quot;') +
        '" style="width:100%;height:320px;border:none;"></iframe>'
    )

    # ---------- 9. logs --------------------------------------------------------------
    if dropped:
        logs = "❌ Dropped locations:\n" + "\n".join(
            f"• {short(locs[n]['name'])} ({demands[n]} t)" for n in dropped
        )
    else:
        logs = "✅ All locations served."

    # ----------10. CSV ---------------------------------------------------------------
    csv_bytes = df_legs.to_csv(index=False).encode("utf-8")

    return totals_html, map_html, csv_bytes, logs

# ---------- B. default params ----------
DEFAULT_PARAMS={
  "fleet":[
    {"type":"7.5t","capacity":7500,"fixed_cost":100,
     "variable_cost_km":1.5,"variable_cost_ton_km":0.5,"count":3},
    {"type":"12t","capacity":12000,"fixed_cost":130,
     "variable_cost_km":1.7,"variable_cost_ton_km":0.6,"count":2}
  ],
  "revenue_per_ton_km":2.5,
  "service_time_per_ton":10,
  "traffic_margin":1.2,
  "working_window_min":480,
  "ors_api_key":"PASTE_KEY"
}

def parse_csv(f):                   # expects name,lat,lon,demand_ton
    df=pd.read_csv(f)
    if not {"name","lat","lon","demand_ton"}.issubset(df.columns):
        raise ValueError("CSV must contain: name,lat,lon,demand_ton")
    return df

In [70]:
# ───────────────────────── ②  GRADIO UI  ─────────────────────────
import gradio as gr, json, datetime, os, pandas as pd, folium, pathlib

blank_table = ""
blank_map   = '<div style="height:500px; background:#f8f8f8;"></div>'

def _path(f):                # returns path or raises
    if f is None: raise ValueError("Upload CSV first.")
    return f["path"] if isinstance(f, dict) else str(f)

def run(pjson, csv_file):
    try:
        tot, mp, csv_bytes, logs = solve_vrp(
            json.loads(pjson), parse_csv(_path(csv_file))
        )
        fname = f"routes_{datetime.datetime.now():%Y%m%d_%H%M%S}.csv"
        out   = f"/content/{fname}"; open(out, "wb").write(csv_bytes)
        return logs, tot, out, mp
    except Exception as e:
        return f"ERROR:\n{e}", blank_table, None, blank_map

with gr.Blocks(title="VRP Demo", theme="default") as demo:
    gr.HTML("""
    <style>
        .flexrow{display:flex; gap:12px}
        .box    {flex:1; border:1px solid #ccc; padding:8px;
                 display:flex; flex-direction:column; justify-content:flex-end;}
    </style>
    """)
    gr.Markdown("### 🚚 VRP Demo")

    with gr.Row(elem_classes="flexrow"):
        # LEFT box  (params + CSV)
        with gr.Column(elem_classes="box"):
            json_box = gr.Code(value=json.dumps(DEFAULT_PARAMS, indent=2),
                               language="json", label="Parameters", lines=10)
            csv_in   = gr.File(label="CSV", file_count="single")

        # RIGHT box (button → logs → table → download)
        with gr.Column(elem_classes="box"):
            run_btn  = gr.Button("Calculate", variant="primary")
            log_box  = gr.Textbox(label="Status", lines=3)
            table_out= gr.HTML(label="Cost summary")
            dl_btn   = gr.File(label="Download legs CSV")

    # full-width, taller map
    map_out = gr.HTML(value=blank_map, label="Map", show_label=True)

    run_btn.click(run,
                  inputs=[json_box, csv_in],
                  outputs=[log_box, table_out, dl_btn, map_out])

demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://19fcda8c164c270f66.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://19fcda8c164c270f66.gradio.live
